In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import copy
import os

In [2]:
os.environ['TORCH_HOME'] = './'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
imsize = 512 if torch.cuda.is_available() else 256
loader = transforms.Compose([
    transforms.Resize((imsize, imsize)),
    transforms.ToTensor(),
])

In [5]:
def load_image(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

In [6]:
content_img = load_image("Cat_1.jpg")
style_img = load_image("The_Great_Wave_off_Kanagawa.jpg")

In [7]:
target_img = content_img.clone()
vgg = models.vgg19(weights=models.VGG19_Weights.DEFAULT).features.to(device).eval()

In [8]:
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # Reshape mean and std to [C, 1, 1] to make them broadcastable
        self.mean = torch.tensor(mean).view(-1, 1, 1).to(device)
        self.std = torch.tensor(std).view(-1, 1, 1).to(device)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std


In [9]:
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

def get_style_model_and_losses(cnn, style_img, content_img):
    cnn = copy.deepcopy(cnn)
    
    # Normalization module
    normalization = Normalization(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    
    content_losses = []
    style_losses = []

    model = nn.Sequential(normalization)

    i = 0  # increment every time we see a conv
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = f'conv_{i}'
        elif isinstance(layer, nn.ReLU):
            name = f'relu_{i}'
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = f'pool_{i}'
        elif isinstance(layer, nn.BatchNorm2d):
            name = f'bn_{i}'
        else:
            raise RuntimeError(f'Unrecognized layer: {layer.__class__.__name__}')

        model.add_module(name, layer)

        if name in content_layers_default:
            # Add content loss:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module(f"content_loss_{i}", content_loss)
            content_losses.append(content_loss)

        if name in style_layers_default:
            # Add style loss:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module(f"style_loss_{i}", style_loss)
            style_losses.append(style_loss)

    # Now we trim off the layers after the last content and style losses
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break
            
    model = model[:(i + 1)]
    
    return model, style_losses, content_losses

# --- 3. Loss Functions ---

class ContentLoss(nn.Module):
    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        self.target = target.detach()
        self.criterion = nn.MSELoss()

    def forward(self, input):
        self.loss = self.criterion(input, self.target)
        return input

def gram_matrix(input):
    batch_size, num_channels, height, width = input.size()
    features = input.view(batch_size * num_channels, height * width)
    G = torch.mm(features, features.t())
    # Normalize by the number of elements in the feature map
    return G.div(batch_size * num_channels * height * width)

class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()
        self.criterion = nn.MSELoss()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = self.criterion(G, self.target)
        return input


In [10]:
target_img.requires_grad_(True)
optimizer = optim.LBFGS([target_img]) # LBFGS is often recommended for NST.

# Set the weights for the content and style loss
style_weight = 1000000
content_weight = 1

unloader = transforms.ToPILImage()

def save_image(tensor, filename="final_output.png"):
    image = tensor.cpu().clone()
    image = image.squeeze(0) # Remove the batch dimension
    image = unloader(image)
    image.save(filename)
    # plt.imshow(image)
    # plt.title("Generated Image")
    # plt.show()

print("Building the style transfer model...")
model, style_losses, content_losses = get_style_model_and_losses(vgg, style_img, content_img)
model.requires_grad_(False) # Freeze the model parameters

print("Optimizing...")
num_steps = 300
run = [0]
while run[0] <= num_steps:

    def closure():
        # Correct the values of updated input image
        target_img.data.clamp_(0, 1)

        optimizer.zero_grad()
        model(target_img)
        
        style_score = 0
        content_score = 0

        for sl in style_losses:
            style_score += sl.loss
        for cl in content_losses:
            content_score += cl.loss

        style_score *= style_weight
        content_score *= content_weight

        loss = style_score + content_score
        loss.backward()

        run[0] += 1
        if run[0] % 50 == 0:
            print(f"run {run[0]}:")
            print(f'Style Loss : {style_score.item():4f} Content Loss: {content_score.item():4f}')
            # Save intermediate images
            save_image(target_img, f'output-{run[0]}.png')

        return style_score + content_score

    optimizer.step(closure)

# --- 5. Final Result ---

# A final clamp to ensure pixel values are valid
target_img.data.clamp_(0, 1)

# Function to convert a tensor to an image and save it
unloader = transforms.ToPILImage()


print("Optimization finished.")
save_image(target_img, "final_output.png")

Building the style transfer model...
Optimizing...
run 50:
Style Loss : 224.224762 Content Loss: 30.040001
run 100:
Style Loss : 63.249813 Content Loss: 30.062893
run 150:
Style Loss : 19.151779 Content Loss: 29.779305
run 200:
Style Loss : 7.359515 Content Loss: 29.224255
run 250:
Style Loss : 3.884834 Content Loss: 28.486897
run 300:
Style Loss : 2.494199 Content Loss: 27.625111
Optimization finished.
